# Установка библиотек

In [2]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=c31b6660ea0f4f08b62e9e920a0c78ca573f09216409f47dbe4887758ff95dc4
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


# Парсинг текста

In [3]:
# Парсинг текста

import re
import os
import pandas as pd

# Функция для извлечения данных из стандартных секций
def extract_sections(content, headers):
    sections = {}
    for i in range(len(headers)):
        header = headers[i]
        if i < len(headers) - 1:
            next_header = headers[i + 1]
            pattern = r'{}(.*?)(?={})'.format(re.escape(header), re.escape(next_header))
        else:
            pattern = r'{}(.*)'.format(re.escape(header))
        match = re.search(pattern, content, flags=re.DOTALL)
        if match:
            sections[header] = match.group(1).strip()
    return sections

# Обновленная функция для извлечения значений из таблицы
def extract_table_content(xml_text):
    # Извлечение строк таблицы
    row_pattern = r'<tr>(.*?)</tr>'
    rows = re.findall(row_pattern, xml_text, re.DOTALL)

    table_info = ""
    for row in rows:
        # Извлечение ячеек таблицы
        cell_pattern = r'<t[hd][^>]*>(?:<content[^>]*>)?(.*?)(?:</content>)?</t[hd]>'
        cells = re.findall(cell_pattern, row, re.DOTALL)
        # Очистка содержимого ячеек от тегов и пробелов
        cleaned_cells = [re.sub(r'<[^>]+>', '', cell).strip() for cell in cells]
        # Пропускаем строки, где ячеек меньше 2 (заголовки или разделители)
        if len(cleaned_cells) >= 2:
            table_info += ', '.join(cleaned_cells) + '\n'
    return table_info

# Функция для извлечения значений из административных полей
def extract_admin_info(xml_text):
    gender_pattern = r'<administrativeGenderCode[^>]*displayName="([^"]+)"'
    gender_match = re.search(gender_pattern, xml_text)

    admin_info = ""
    if gender_match:
        gender = gender_match.group(1)
        admin_info += f"Пол: {gender}\n"

    return admin_info

# Функция для удаления ненужных тегов и HTML-сущностей
def clean_text(text):
    # Удаляем теги <title>, <text>, и подобные
    text = re.sub(r'<title>|</title>|<text>|</text>', '', text, flags=re.DOTALL)

    # Заменяем HTML сущности
    text = re.sub(r'&amp;', '&', text)
    text = re.sub(r'&quot;', '"', text)
    text = re.sub(r'&lt;', '<', text)
    text = re.sub(r'&gt;', '>', text)

    # Удаляем любые оставшиеся HTML теги
    text = re.sub(r'<[^>]+>', '', text)

    # Очищаем лишние пробелы и пустые строки
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Основная функция для извлечения информации из медицинских карт
def extract_medical_info(xml_text):
    # Извлекаем содержимое между тегами <value>...</value>
    pattern = r'<value[^>]*>(.*?)</value>'
    matches = re.findall(pattern, xml_text, re.DOTALL)

    medical_info = ""

    # Определяем важные разделы
    headers = [
        'Возраст (дата рождения):',
        'Дата поступления:',
        'Этап медицинской реабилитации:',
        'Жалобы',
        'Анамнез заболевания',
        'Объективный статус:',
        'Результаты специальных методов исследования:',
        'Клинический диагноз:',
        'Оценка реабилитационного статуса пациента:',
        'Шкалы оценки',
        'Факторы, ограничивающие проведение реабилитационных мероприятий:',
        'Реабилитационный потенциал:',
    ]

    # Извлечение информации по стандартным секциям
    for match in matches:
        content = match

        # Удаляем раздел "Индивидуальная программа реабилитации" и все, что после него
        content = re.sub(r'Индивидуальная программа реабилитации:.*', '', content, flags=re.DOTALL)

        # Извлекаем необходимые разделы
        sections = extract_sections(content, headers)

        # Собираем медицинскую информацию из извлеченных разделов
        for header in headers:
            if header in sections:
                medical_info += header + '\n' + sections[header].strip() + '\n\n'

    # Извлекаем данные из таблиц
    table_info = extract_table_content(xml_text)
    if table_info:
        medical_info += f"Табличные данные:\n{table_info}\n"

    # Извлекаем административные данные, такие как пол пациента
    admin_info = extract_admin_info(xml_text)
    if admin_info:
        medical_info += f"Административные данные:\n{admin_info}\n"

    # Удаляем дубликаты строк в результате
    medical_info = "\n".join(dict.fromkeys(medical_info.splitlines()))

    # Очищаем текст от ненужных тегов и сущностей
    medical_info = clean_text(medical_info)

    return medical_info

In [4]:
# Укажите путь к директории
directory = '/content/drive/MyDrive/DocsNew3'

# Список для хранения результатов по всем файлам
all_medical_info = []

# Проходим по всем XML-файлам в директории
for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            xml_text = f.read()

        medical_info = extract_medical_info(xml_text)
        all_medical_info.append({'id': filename, 'text': medical_info})

# Создаем DataFrame из списка словарей
df = pd.DataFrame(all_medical_info)

In [5]:
print(df['text'][0])

Табличные данные: Показатель, Значение / чувствительность (для бак.исследований), Единицы измерения, Референтный диапазон, Комментарий, Оборудование, Дата, Исполнитель Нейтрофилы, абсолютное количество в крови методом автоматизированного подсчёта, 5.18, 10^9/л, 2 - 5.8 10^9/л, Sysmex4000_1, 18.08.2023 10:10, Биолог К Л.В. Нейтрофилы, относительное количество в крови методом автоматизированного подсчёта, 62, %, 47 - 76 %, , 18.08.2023 10:10, Биолог К Л.В. Лимфоциты, абсолютное количество в крови методом автоматизированного подсчёта, 2.13, 10^9/л, 1.2 - 3 10^9/л, , 18.08.2023 10:10, Биолог К Л.В. Лейкоциты в крови, количество, скорректированное на количество ядросодержащих эритроцитов, методом автоматизированного подсчёта, 8.38, 10^9/л, 4.5 - 9 10^9/л, Sysmex4000_1, 18.08.2023 10:10, Биолог К Л.В. Лимфоциты, относительное количество в крови методом автоматизированного подсчёта, 25, %, 19 - 45 %, , 18.08.2023 10:10, Биолог К Л.В. Моноциты, абсолютное количество в крови методом автоматизир

# Дополнительная обработка и удаление ненужной информации

In [6]:
# Дополнительная обработка и удаление ненужной информации

# Импорт необходимых библиотек
import nltk
from collections import Counter

# Если необходимо, скачиваем ресурсы NLTK
nltk.download('punkt')

# Функция обезличивания текста
def anonymize_text(text):
    # Замена персональных данных на placeholders
    text = re.sub(r'\bФ\.И\.О\.\s*[:\-]*\s*([\w\s\.]+)', 'ФИО пациента', text)
    text = re.sub(r'\bНомер истории болезни\s*[:\-]*\s*([\w\d]+)', 'Номер истории болезни', text)
    text = re.sub(r'\bАдрес\s*[:\-]*\s*([\w\s\.\,]+)', 'Адрес пациента', text)
    text = re.sub(r'\bДата рождения\s*[:\-]*\s*([\d\.]+)', 'Дата рождения пациента', text)
    text = re.sub(r'!\s*!+', '', text)  # Удаление лишних восклицательных знаков
    return text

# Функция удаления повторяющейся информации
def remove_repetitive_info(text):
    # Разбиваем текст на строки
    lines = text.split('\n')
    seen = set()
    result = []

    # Ключевые слова, по которым мы будем определять важные строки
    important_keywords = ['анализ', 'результаты', 'крови', 'биохимия', 'общий анализ', 'клинический анализ', 'гемоглобин', 'лейкоциты']

    for line in lines:
        # Если строка содержит важные ключевые слова, всегда сохраняем её
        if any(keyword in line.lower() for keyword in important_keywords):
            result.append(line)
            continue

        # Если строка не встречалась ранее, добавляем её
        if line not in seen:
            result.append(line)
            seen.add(line)
        else:
            # Иначе пропускаем повторяющуюся строку
            pass  # Можно добавить логирование при необходимости

    # Объединяем обратно в текст
    return '\n'.join(result)

# Функция очистки текста от ненужных символов и пробелов
def clean_text(text):
    # Удаляем специфические ненужные символы, не затрагивая цифры и знаки, используемые в анализах
    text = re.sub(r'[^\w\s\.\,\:\;\-\^\%\(\)\/\+\=]', ' ', text)
    # Удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
new_text = []

# Проходим по всем тестам в DataFrame
for text in df['text'].values:
  text = anonymize_text(text)
  text = remove_repetitive_info(text)
  text = clean_text(text)
  new_text.append(text)

#Создаем новое поле с дополнительно обработанным текстом
df['new_text'] = new_text

In [8]:
# Вывод обработанного текста
print(df['new_text'][0])

Табличные данные: Показатель, Значение / чувствительность (для бак.исследований), Единицы измерения, Референтный диапазон, Комментарий, Оборудование, Дата, Исполнитель Нейтрофилы, абсолютное количество в крови методом автоматизированного подсчёта, 5.18, 10^9/л, 2 - 5.8 10^9/л, Sysmex4000_1, 18.08.2023 10:10, Биолог К Л.В. Нейтрофилы, относительное количество в крови методом автоматизированного подсчёта, 62, %, 47 - 76 %, , 18.08.2023 10:10, Биолог К Л.В. Лимфоциты, абсолютное количество в крови методом автоматизированного подсчёта, 2.13, 10^9/л, 1.2 - 3 10^9/л, , 18.08.2023 10:10, Биолог К Л.В. Лейкоциты в крови, количество, скорректированное на количество ядросодержащих эритроцитов, методом автоматизированного подсчёта, 8.38, 10^9/л, 4.5 - 9 10^9/л, Sysmex4000_1, 18.08.2023 10:10, Биолог К Л.В. Лимфоциты, относительное количество в крови методом автоматизированного подсчёта, 25, %, 19 - 45 %, , 18.08.2023 10:10, Биолог К Л.В. Моноциты, абсолютное количество в крови методом автоматизир

# NLP обработка

In [9]:
# Импорт необходимых библиотек
import pandas as pd
import nltk
import pymorphy2
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

# Инициализация инструментов
morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

# Функция для подготовки текста для BERT
def prepare_for_bert(df, text_column):
    """
    Функция принимает DataFrame и название столбца с текстом.
    Возвращает DataFrame с новым столбцом 'bert_text' для ввода в модель BERT.
    """
    # Для BERT минимальная предобработка: удаление лишних пробелов
    df['bert_text'] = df[text_column].apply(lambda x: ' '.join(x.split()))
    return df[['id', 'bert_text']]

# Функция для подготовки текста для других моделей
def prepare_for_other_models(df, text_column):
    """
    Функция принимает DataFrame и название столбца с текстом.
    Возвращает DataFrame с новым столбцом 'processed_text' после NLP обработки.
    """
    # Функция для токенизации, лемматизации и удаления стоп-слов
    def preprocess_text(text):
        # Токенизация
        tokens = word_tokenize(text, language='russian')
        # Приведение к нижнему регистру и удаление неалфавитных символов
        tokens = [token.lower() for token in tokens if token.isalpha()]
        # Удаление стоп-слов и лемматизация
        tokens = [morph.normal_forms(token)[0] for token in tokens if token not in stop_words]
        # Возвращаем обработанный текст
        return ' '.join(tokens)

    df['processed_text'] = df[text_column].apply(preprocess_text)
    return df[['id', 'processed_text']]

# Дополнительно: Функция для вычисления TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

def compute_tfidf(df, text_column):
    """
    Функция принимает DataFrame и название столбца с обработанным текстом.
    Возвращает матрицу TF-IDF и объект Vectorizer.
    """
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df[text_column])
    return tfidf_matrix, vectorizer

# Дополнительно: Функция для обучения модели Word2Vec
from gensim.models import Word2Vec

def train_word2vec(df, text_column):
    """
    Функция принимает DataFrame и название столбца с обработанным текстом.
    Возвращает обученную модель Word2Vec.
    """
    # Преобразуем тексты в списки токенов
    tokenized_texts = df[text_column].apply(lambda x: x.split())
    # Обучаем модель Word2Vec
    w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)
    return w2v_model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
# Подготовка данных для BERT
df_bert = prepare_for_bert(df.copy(), 'new_text')
print("Данные для BERT:")

# Подготовка данных для других моделей
df_other = prepare_for_other_models(df.copy(), 'new_text')
print("\nДанные для других моделей:")
print(df_other)

# Вычисление TF-IDF
tfidf_matrix, vectorizer = compute_tfidf(df_other, 'processed_text')
print("\nTF-IDF матрица:")
print(tfidf_matrix.toarray())

# Обучение модели Word2Vec
w2v_model = train_word2vec(df_other, 'processed_text')
print("\nПример векторного представления слова 'пациент':")
if 'пациент' in w2v_model.wv:
    print(w2v_model.wv['пациент'])
else:
    print("Слово 'пациент' отсутствует в словаре модели Word2Vec.")

Данные для BERT:

Данные для других моделей:
                                   id  \
0     EMD_LAB_188219728_255580489.xml   
1     EMD_LAB_188198466_255586001.xml   
2     EMD_LAB_188198466_255587038.xml   
3     EMD_LAB_188198466_255587914.xml   
4     EMD_LAB_188219617_255578360.xml   
...                               ...   
4207  EMD_LAB_187249519_255545720.xml   
4208  EMD_LAB_187249519_255545678.xml   
4209  EMD_LAB_187273537_255558885.xml   
4210  EMD_LAB_187277604_255547019.xml   
4211  EMD_LAB_187273457_255545101.xml   

                                         processed_text  
0     табличный дать показатель значение чувствитель...  
1     табличный дать показатель значение чувствитель...  
2     табличный дать показатель значение чувствитель...  
3     табличный дать показатель значение чувствитель...  
4     табличный дать показатель значение чувствитель...  
...                                                 ...  
4207  табличный дать показатель значение чувствитель... 

# Получение численных значений анализов

In [13]:
import re
import pandas as pd

# Список интересующих нас анализов
target_tests = [
    'Лейкоциты', 'Нейтрофилы', 'Лимфоциты', 'Моноциты',
    'Эозинофилы', 'Базофилы', 'Эритроциты', 'Гемоглобин',
    'Гематокрит', 'Тромбоциты', 'Скорость оседания эритроцитов'
]

# Функция для извлечения результатов интересующих анализов
def parse_blood_tests(text):
    """
    Функция для быстрого парсинга результатов определенных анализов из текста.

    Параметры:
    text (str): строка с результатами анализов.

    Возвращает:
    results_dict (dict): словарь с названиями анализов и их значениями.
    """
    results_dict = {}
    # Разбиваем текст на строки
    lines = text.split('\n')
    for line in lines:
        # Проверяем, содержит ли строка интересующий нас анализ
        for test in target_tests:
            if test in line:
                # Ищем название анализа и значение
                match = re.search(rf'{test}.*?,\s*([\d\.]+)', line)
                if match:
                    value = match.group(1)
                    results_dict[test] = value
    return results_dict

def process_dataframe(df):
    """
    Функция для обработки DataFrame с полями 'id' и 'text', извлечения результатов анализов и формирования нового DataFrame.

    Параметры:
    df (DataFrame): исходный DataFrame с полями 'id' и 'text'.

    Возвращает:
    df_results (DataFrame): результирующий DataFrame, где строки — это id, колонки — признаки, значения — результаты анализов.
    """
    results_list = []
    for index, row in df.iterrows():
        id_value = row['id']
        text = row['new_text']
        # Извлекаем результаты анализов в виде словаря
        lab_results = parse_blood_tests(text)
        lab_results['id'] = id_value  # Добавляем id в словарь
        results_list.append(lab_results)

    # Преобразуем список словарей в DataFrame
    df_results = pd.DataFrame(results_list)
    # Устанавливаем 'id' в качестве индекса
    df_results.set_index('id', inplace=True)
    return df_results

# Пример использования
# Предположим, что у вас есть DataFrame df с колонками 'id' и 'text'

# df = pd.DataFrame({'id': ['patient_1'], 'text': [your_text_here]})

In [16]:
df_results = process_dataframe(df)

In [17]:
df_results

,Лейкоциты,Нейтрофилы,Лимфоциты,Моноциты,Эозинофилы,Базофилы,Эритроциты,Гемоглобин,Гематокрит,Тромбоциты,Скорость оседания эритроцитов
id,,,,,,,,,,,
EMD_LAB_188219728_255580489.xml,8.38,5.18,2.13,0.84,0.19,0.04,4.84,141,41.8,210,20
EMD_LAB_188198466_255586001.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EMD_LAB_188198466_255587038.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EMD_LAB_188198466_255587914.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EMD_LAB_188219617_255578360.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
EMD_LAB_187249519_255545720.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102,31.4,NaN,NaN
EMD_LAB_187249519_255545678.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EMD_LAB_187273537_255558885.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Количество обработанных медкарт

In [19]:
print(len(df))

4212


In [20]:
print(len(df_results))

4212
